In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import os

# 1. Preprocessing & Data Augmentation
IMG_SIZE = (160, 160)
BATCH_SIZE = 32

# Creating a synthetic dataset for demonstration
def create_dummy_data():
    X = np.random.rand(100, 160, 160, 3).astype(np.float32)
    y = np.random.randint(0, 2, 100) # 0: Organic, 1: Recyclable
    return X, y

train_X, train_y = create_dummy_data()

print("Dataset ready for training.")

Dataset ready for training.


In [3]:
# Load MobileNetV2 without the heavy top layers
base_model = tf.keras.applications.MobileNetV2(input_shape=(160, 160, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False # Freeze the base

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid') # Binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train for 5 epochs
history = model.fit(train_X, train_y, epochs=5, batch_size=BATCH_SIZE)

Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 349ms/step - accuracy: 0.5000 - loss: 0.7261
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 356ms/step - accuracy: 0.4800 - loss: 0.7263
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 334ms/step - accuracy: 0.5000 - loss: 0.7136
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 340ms/step - accuracy: 0.5100 - loss: 0.7058
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 329ms/step - accuracy: 0.5800 - loss: 0.6880


In [4]:
# 1. Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# 2. Apply dynamic range quantization (reduces size by ~4x)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# 3. Save the model
with open('recycling_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model successfully converted to recycling_model.tflite")
print(f"TFLite Model Size: {len(tflite_model) / 1024:.2f} KB")

INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp_quv_ec8\assets


INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp_quv_ec8\assets


Saved artifact at 'C:\Users\User\AppData\Local\Temp\tmp_quv_ec8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 160, 160, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2173161417104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2171751637904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2171751637712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2171751638288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2171751637520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2171751638480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2171751637136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2171751639248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2171751638096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2171751636368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  21

In [5]:
# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="recycling_model.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test on one random image
input_shape = input_details[0]['shape']
input_data = np.expand_dims(train_X[0], axis=0)

interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
print(f"Prediction (0=Organic, 1=Recyclable): {output_data[0][0]}")

Prediction (0=Organic, 1=Recyclable): 0.6335718631744385


c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


**How Edge AI benefits real-time applications:**

**_(a) Ultra-Low Latency:_** above, the inference happens locally. In a real-world sorting plant, the robot can identify trash in 5–20 milliseconds because it doesn't have to wait for a 5G/Wi-Fi signal to send an image to a server in another city and wait for the result to come back.

**_(b) Bandwidth Efficiency:_** Instead of streaming 24/7 high-definition video to the cloud (which is expensive and slows down the network), only the final "result" (e.g. "It's a bottle") is processed.

**_(c)Reliability:_** Even if the facility's internet goes down, the sorting line keeps moving because the "brain" (the .tflite model) is physically on the machine.

**_(d)Privacy:_** If the camera captures sensitive data (like a person's face or an address on a discarded envelope), that data stays on the local device and is never uploaded to the cloud.
